In [1]:
# The following section is used to generate information about the datasets so these 
# variables can be initialised in the CNN Model. The information gathered here will
# also be used to determine an appropiate size of the training and testing datasets.
import os                                                   # Import os to change the information surrounding the enviroment 
from keras.preprocessing.image import ImageDataGenerator    # Create tensorflow image batches using data augmentation.
os.environ['TF_XLA_FLAGS'] = '--tf_xla_enable_xla_devices'
locationOfDataset = '/home/dmr/DatasetForExperiments/scareware/'
batches = ImageDataGenerator().flow_from_directory(directory=locationOfDataset, target_size=(64,64), batch_size=1010)

# Print information about Images 
imgs, labels = next(batches)
print("Number of Images and Size of Images: ",imgs.shape) # Images with width x length x depth
print("Number of Images and Families: ",labels.shape) # Lablels with batch_size, number of classes

2023-05-05 15:04:12.127348: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


Found 97 images belonging to 11 classes.
Number of Images and Size of Images:  (97, 64, 64, 3)
Number of Images and Families:  (97, 11)


In [2]:
# This section will cover the necessary steps used to split the scareware dataset
# into training and testing to evaluate the effectiveness of this convolutional neural 
# network model based on the scareware samples.
import numpy as np                                   # Provides the ability to use numerical data to convert the .dex files to greyscale images
import scipy as sp                                   # Provides the use of algorithms to be used 
from PIL import Image                                # Allow for processing of images

# Splitting the Dataset into Training and testing with a test size of 30% of overall 
# samples in the dataset. 
from sklearn.model_selection import train_test_split    # Used to split the dataset

# Split up the data set, 10% testing 90% training
trainX, testX, trainY, testY = train_test_split(imgs/255.,labels, test_size=0.2)

# Print the information regarding the training and testing sample sizes
#print("Size of Training Set: ", trainX.shape)                   # Print training set information
#print("Size of Testing Set: ",testX.shape)                      # Print testing set information
print("Number of Samples in Training Set: ",trainY.shape)   # Print training set information
print("Number of Samples in Testing Set: ",testY.shape)     # Print testing set information

Number of Samples in Training Set:  (77, 11)
Number of Samples in Testing Set:  (20, 11)


In [3]:
# The Convolutional Neural Network Model
# This section will encompass the aspects surrounding the convultional neural network
# which was used for the experimentation of this research. 
import keras                                            # Import keras
import tensorflow                                       # Import tensorflow
from keras.models import Sequential, Input, Model       # Used for CNN model regarding Models
from keras.layers import Dense, Dropout, Flatten        # Used for CNN model regarding Layers
from keras.layers import Conv2D, MaxPooling2D           # Used for CNN model regarding Layers
from tensorflow.keras.layers import BatchNormalization  # Layer used to normalise inputs
from tensorflow.keras.metrics import AUC                # Used estimate to Area Under Curve

noOfFamilies = 11 # Number of Families in the Sample

# Function used to create the layers of the convolutional neural network model. These parameters were
# constantly changed throughout testing in order to obtain the best possible results. Within this model 
# there are 10 main layers and each layers parameters are explained in detail. 
def CNNModel():
    CNNModel = Sequential()        # Sequential Model is used the experiments

    # Convolutional Layer: With relu Activation, 31 filters, and a kernel size of 3 x 3
    CNNModel.add(Conv2D(30, kernel_size=(3, 3),
                     activation='relu',
                     input_shape=(64,64,3)))                # Layer 1 

    # Max Pooling Layer: With a pool size of 2 x 2
    CNNModel.add(MaxPooling2D(pool_size=(2, 2)))            # Layer 2
    
    # Convolutional Layer: 17 filters and kernel size of 3 x 3 
    CNNModel.add(Conv2D(17, (3, 3), activation='relu'))     # Layer 3
    
    # Max Pooling Layer: Pool size of 2 x 2 
    CNNModel.add(MaxPooling2D(pool_size=(2, 2)))            # Layer 4
    
    # Drop Out Layer: This layer used to drop 25% of neurons in the model .
    CNNModel.add(Dropout(0.25))                             # Layer 5
    
    # Flatten Layer
    CNNModel.add(Flatten())                                 # Layer 6
    
    # Dense/Fully Connected Layer : 128 Neurons using the relu activation function
    CNNModel.add(Dense(128, activation='relu'))             # Layer 7
    
    # Drop Out Layer: This layer used to drop 50% of neurons in the model.
    CNNModel.add(Dropout(0.5))                              # Layer 8
    
    # Dense/Fully Connected Layer : 50 Neurons using the relu activation function
    CNNModel.add(Dense(50, activation='relu'))              # Layer 9
    
    # Dense/Fully Connected Layer : number of families neurons, using the softmax activation function
    CNNModel.add(Dense(noOfFamilies, activation='softmax')) # Layer 10
    
    CNNModel.compile(loss='categorical_crossentropy', optimizer = 'adam', metrics=['accuracy'])
    return CNNModel     # Return the Convolutional Neural Network Model

In [4]:
# Measure the effectiveness of the model. This section is used to generate the results of the 
# corresponding convolutional neural network. For the purpose of this mode, there are three main 
# measurements which have to be declared. These are recall, preicision and f1. Listed below
# are the neccessary formulas which are required to obtain these results.
from keras import backend as K

# Recall Measurement
def recallMeasurement(testY, yPrediction):
    true_positives = K.sum(K.round(K.clip(testY * yPrediction, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(testY, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

# Precision Measurement
def precisionMeasurement(testY, yPrediction):
    true_positives = K.sum(K.round(K.clip(testY * yPrediction, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(yPrediction, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

# F1 Measurement
def f1Measurement(testY, yPrediction):
    precision = precisionMeasurement(testY, yPrediction)
    recall = recallMeasurement(testY, yPrediction)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [5]:
# Initialise the Model
CNNModel = CNNModel()

2023-05-05 15:04:15.300707: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2023-05-05 15:04:15.619438: E tensorflow/stream_executor/cuda/cuda_driver.cc:328] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-05-05 15:04:15.619540: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (pop-os): /proc/driver/nvidia/version does not exist
2023-05-05 15:04:15.620765: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-05 15:04:15.633103: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 1999965000 Hz
2023-05-05 15:04:15.6343

In [6]:
# Print the Information of the Model. This includes the information surrounding 
# the layers of the models as well as the number of total parameters
CNNModel.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 62, 62, 30)        840       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 31, 31, 30)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 29, 29, 17)        4607      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 17)        0         
_________________________________________________________________
dropout (Dropout)            (None, 14, 14, 17)        0         
_________________________________________________________________
flatten (Flatten)            (None, 3332)              0         
_________________________________________________________________
dense (Dense)                (None, 128)               4

In [7]:
trainY.shape                            # Returns the shape of the trainY array 
trainY_new = np.argmax(trainY, axis=1)  # Converts the trainY array to an array of integers 
trainY_new                              # Returns the array of integer labels

array([ 7,  4,  6,  6,  9,  4,  6,  9,  4,  1,  5,  5, 10,  7, 10,  4,  1,
        3,  4,  5,  4, 10,  2,  3,  9,  9,  4,  4,  5, 10,  5,  9,  6, 10,
        7,  9,  6,  5,  7,  2, 10,  8,  1,  1,  9,  3,  2,  8,  5,  6,  3,
        2,  3,  1,  9,  3,  0,  7,  9,  0,  8,  6,  9,  1,  2,  5,  4,  8,
        6, 10,  6,  7,  3,  8,  3,  1,  6])

In [8]:
# This section of the model imports the class_weight function from the sklearn.utils 
# library. This is used to compute the class weights for a multi-class classification problem.
# The weightOfModel computes the class weights based on the class distribution found in the 
# training set. The class_weight parameter is set to balanced which means that the class 
# weights are automatically adjusted in acorrdance with the information obtained from the 
# input data.

from sklearn.utils import class_weight # Used to compute the class weights for a multi-class classification system
weightOfModel = class_weight.compute_class_weight(class_weight = 'balanced',
                                                 classes = np.unique(trainY_new),
                                                 y = trainY_new)

# This code is used to map each unique class label in the training set trainY_new variable. 
# This is then uses the class_weight variable initialised above. 
weightOfModel = {l:c for l,c in zip(np.unique(trainY_new), weightOfModel)}

In [9]:
# This is used to compile the Convolutional Neural network Model. It uses the adam optimiser, 
# computes loss using binary_crossentropy and then calculates the measurements listed before.
CNNModel.compile(optimizer='adam', 
                    loss='binary_crossentropy', 
                    metrics=[f1Measurement,precisionMeasurement, 
                    recallMeasurement,
                    AUC()])

In [10]:
# This section of the model is used to trained the convolutional neural network model. 
# This is achieved by using the fit() method from the Keras API of the TensorFlow library which 
# was imported previously. Here is where the number of epochs are declared for the model also
CNNModel.fit(trainX, trainY, validation_data=(testX, testY), 
                epochs=150,  
                class_weight=weightOfModel)

2023-05-05 15:04:15.821353: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)


Epoch 1/150
3/3 [==============================] - 1s 282ms/step - loss: 0.6472 - f1Measurement: 0.0000e+00 - precisionMeasurement: 0.0000e+00 - recallMeasurement: 0.0000e+00 - auc: 0.4848 - val_loss: 0.3847 - val_f1Measurement: 0.0000e+00 - val_precisionMeasurement: 0.0000e+00 - val_recallMeasurement: 0.0000e+00 - val_auc: 0.4945
Epoch 2/150
3/3 [==============================] - 0s 53ms/step - loss: 0.4390 - f1Measurement: 0.0625 - precisionMeasurement: 0.1621 - recallMeasurement: 0.0389 - auc: 0.5261 - val_loss: 0.3649 - val_f1Measurement: 0.0588 - val_precisionMeasurement: 0.0714 - val_recallMeasurement: 0.0500 - val_auc: 0.4924
Epoch 3/150
3/3 [==============================] - 0s 53ms/step - loss: 0.3917 - f1Measurement: 0.0638 - precisionMeasurement: 0.1012 - recallMeasurement: 0.0467 - auc: 0.5367 - val_loss: 0.3479 - val_f1Measurement: 0.0000e+00 - val_precisionMeasurement: 0.0000e+00 - val_recallMeasurement: 0.0000e+00 - val_auc: 0.5271
Epoch 4/150
3/3 [======================

3/3 [==============================] - 0s 57ms/step - loss: 0.2417 - f1Measurement: 0.1343 - precisionMeasurement: 1.0000 - recallMeasurement: 0.0727 - auc: 0.8679 - val_loss: 0.2691 - val_f1Measurement: 0.0952 - val_precisionMeasurement: 1.0000 - val_recallMeasurement: 0.0500 - val_auc: 0.7616
Epoch 27/150
3/3 [==============================] - 0s 58ms/step - loss: 0.2466 - f1Measurement: 0.1704 - precisionMeasurement: 0.8333 - recallMeasurement: 0.0951 - auc: 0.8284 - val_loss: 0.2751 - val_f1Measurement: 0.0952 - val_precisionMeasurement: 1.0000 - val_recallMeasurement: 0.0500 - val_auc: 0.7284
Epoch 28/150
3/3 [==============================] - 0s 63ms/step - loss: 0.2371 - f1Measurement: 0.2062 - precisionMeasurement: 0.5712 - recallMeasurement: 0.1276 - auc: 0.8669 - val_loss: 0.2757 - val_f1Measurement: 0.0952 - val_precisionMeasurement: 1.0000 - val_recallMeasurement: 0.0500 - val_auc: 0.7248
Epoch 29/150
3/3 [==============================] - 0s 63ms/step - loss: 0.2263 - f1Me

Epoch 53/150
3/3 [==============================] - 0s 56ms/step - loss: 0.0945 - f1Measurement: 0.8587 - precisionMeasurement: 0.9644 - recallMeasurement: 0.7751 - auc: 0.9902 - val_loss: 0.2153 - val_f1Measurement: 0.5625 - val_precisionMeasurement: 0.7500 - val_recallMeasurement: 0.4500 - val_auc: 0.8607
Epoch 54/150
3/3 [==============================] - 0s 52ms/step - loss: 0.0809 - f1Measurement: 0.9199 - precisionMeasurement: 0.9660 - recallMeasurement: 0.8780 - auc: 0.9958 - val_loss: 0.2254 - val_f1Measurement: 0.6250 - val_precisionMeasurement: 0.8333 - val_recallMeasurement: 0.5000 - val_auc: 0.8723
Epoch 55/150
3/3 [==============================] - 0s 52ms/step - loss: 0.0860 - f1Measurement: 0.8784 - precisionMeasurement: 0.9039 - recallMeasurement: 0.8561 - auc: 0.9922 - val_loss: 0.2278 - val_f1Measurement: 0.6286 - val_precisionMeasurement: 0.7333 - val_recallMeasurement: 0.5500 - val_auc: 0.8944
Epoch 56/150
3/3 [==============================] - 0s 52ms/step - loss: 

3/3 [==============================] - 0s 50ms/step - loss: 0.0330 - f1Measurement: 0.9692 - precisionMeasurement: 0.9872 - recallMeasurement: 0.9520 - auc: 0.9999 - val_loss: 0.2181 - val_f1Measurement: 0.6286 - val_precisionMeasurement: 0.7333 - val_recallMeasurement: 0.5500 - val_auc: 0.9170
Epoch 80/150
3/3 [==============================] - 0s 50ms/step - loss: 0.0369 - f1Measurement: 0.9820 - precisionMeasurement: 1.0000 - recallMeasurement: 0.9648 - auc: 0.9999 - val_loss: 0.2110 - val_f1Measurement: 0.6316 - val_precisionMeasurement: 0.6667 - val_recallMeasurement: 0.6000 - val_auc: 0.9195
Epoch 81/150
3/3 [==============================] - 0s 51ms/step - loss: 0.0292 - f1Measurement: 0.9831 - precisionMeasurement: 0.9831 - recallMeasurement: 0.9831 - auc: 0.9998 - val_loss: 0.2252 - val_f1Measurement: 0.6154 - val_precisionMeasurement: 0.6316 - val_recallMeasurement: 0.6000 - val_auc: 0.9199
Epoch 82/150
3/3 [==============================] - 0s 53ms/step - loss: 0.0268 - f1Me

3/3 [==============================] - 0s 51ms/step - loss: 0.0337 - f1Measurement: 0.9479 - precisionMeasurement: 0.9479 - recallMeasurement: 0.9479 - auc: 0.9989 - val_loss: 0.2728 - val_f1Measurement: 0.5946 - val_precisionMeasurement: 0.6471 - val_recallMeasurement: 0.5500 - val_auc: 0.8775
Epoch 106/150
3/3 [==============================] - 0s 50ms/step - loss: 0.0204 - f1Measurement: 1.0000 - precisionMeasurement: 1.0000 - recallMeasurement: 1.0000 - auc: 1.0000 - val_loss: 0.2832 - val_f1Measurement: 0.6111 - val_precisionMeasurement: 0.6875 - val_recallMeasurement: 0.5500 - val_auc: 0.8519
Epoch 107/150
3/3 [==============================] - 0s 51ms/step - loss: 0.0160 - f1Measurement: 1.0000 - precisionMeasurement: 1.0000 - recallMeasurement: 1.0000 - auc: 1.0000 - val_loss: 0.3037 - val_f1Measurement: 0.6111 - val_precisionMeasurement: 0.6875 - val_recallMeasurement: 0.5500 - val_auc: 0.8532
Epoch 108/150
3/3 [==============================] - 0s 51ms/step - loss: 0.0117 - f

3/3 [==============================] - 0s 58ms/step - loss: 0.0158 - f1Measurement: 0.9861 - precisionMeasurement: 1.0000 - recallMeasurement: 0.9744 - auc: 0.9999 - val_loss: 0.2738 - val_f1Measurement: 0.6316 - val_precisionMeasurement: 0.6667 - val_recallMeasurement: 0.6000 - val_auc: 0.8829
Epoch 132/150
3/3 [==============================] - 0s 61ms/step - loss: 0.0080 - f1Measurement: 0.9831 - precisionMeasurement: 0.9831 - recallMeasurement: 0.9831 - auc: 1.0000 - val_loss: 0.2754 - val_f1Measurement: 0.6154 - val_precisionMeasurement: 0.6316 - val_recallMeasurement: 0.6000 - val_auc: 0.8822
Epoch 133/150
3/3 [==============================] - 0s 56ms/step - loss: 0.0162 - f1Measurement: 0.9954 - precisionMeasurement: 1.0000 - recallMeasurement: 0.9909 - auc: 1.0000 - val_loss: 0.2825 - val_f1Measurement: 0.6000 - val_precisionMeasurement: 0.6000 - val_recallMeasurement: 0.6000 - val_auc: 0.9040
Epoch 134/150
3/3 [==============================] - 0s 57ms/step - loss: 0.0104 - f

In [11]:
# Evaluates the trained  model on the test data and returns the values of the  metrics
scores = CNNModel.evaluate(testX, testY) 
print("--------------------------------------------------------------------------------------------------------------------")

# Evaluates the trained model on the test data and declares the loss, accuracy 
# f1_score, precision and recall
loss, accuracy, f1_score, precision, recall = CNNModel.evaluate(testX, testY)
print("--------------------------------------------------------------------------------------------------------------------")

# Used to Print the Final CNN Accuracy of the model
print("Final Results for The CNN Model")
print("F1 Accuracy of: {:.2f}".format(scores[1]),'%')    # Print Accuracy of Model
print("With a Loss of: {:.2f}".format(scores[0]),'%')    # Print Loss of Model

1/1 [==============================] - 0s 24ms/step - loss: 0.3346 - f1Measurement: 0.6842 - precisionMeasurement: 0.7222 - recallMeasurement: 0.6500 - auc: 0.8789
--------------------------------------------------------------------------------------------------------------------
1/1 [==============================] - 0s 23ms/step - loss: 0.3346 - f1Measurement: 0.6842 - precisionMeasurement: 0.7222 - recallMeasurement: 0.6500 - auc: 0.8789
--------------------------------------------------------------------------------------------------------------------
Final Results for The CNN Model
F1 Accuracy of: 0.68 %
With a Loss of: 0.33 %
